# Problem
Our problem is a classification problem, our target is whether a shot is goal or not.

# Where Can We Use This Model In Real World ?

We can use this model to analyze player performances, understand player habits on the field, determining which players to change in game or find out in advance whether a new player will be compatible with the team or not during transfer seasons.

# Imports

In [ ]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import rpy2.robjects.packages as rpackages
from rpy2.robjects.conversion import localconverter
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
import h2o
from h2o.automl import H2OAutoML
import pickle

In [ ]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

# Getting Dataset Over R from Python

In [ ]:
pandas2ri.activate()
ro.r('''
        library("worldfootballR")
        laliga <- load_understat_league_shots(league = "La liga")
     ''')
laliga = pandas2ri.rpy2py(ro.r['laliga'])
laliga.drop('league', axis=1, inplace=True)
data = laliga[(laliga['date'] > '2022-01-01') & (laliga['date'] < '2024-06-10')]


We are going to use La Liga dataset between 2023-01-01 and 2024-06-10.

# Data Manipulations and Fixes

We have a problem about NaN values and duplicate features, we are going to fix these problems by manipulating the data.

In [ ]:
warnings.filterwarnings('ignore')

def fixDataNaN(df):
    with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.py2rpy(df)
pairs = [['x','X'],['y','Y'],['x_g','xG'],['h_a','home_away'],['shot_type','shotType'],['last_action','lastAction']]

def camel_case_columns(df):
    def camel_case(column_name):
        parts = column_name.split('_')
        return str(parts[0] + ''.join(x.title() for x in parts[1:]))
    
    new_columns = []
    for column in df.columns:
        if '_' in column:
            new_columns.append(camel_case(column))
        else:
            new_columns.append(str(column))
    
    df.columns = new_columns
    return df

def fixMergeColumns(dataList, pairs):
    for targetData in dataList:
        for pair in pairs:
            if pair[0] in targetData.columns and pair[1] in targetData.columns:
                targetData['{}'.format(pair[1])].fillna(targetData['{}'.format(pair[0])], inplace=True)
                targetData.drop(columns=['{}'.format(pair[0])], inplace=True)
        targetData = camel_case_columns(targetData)
        fixDataNaN(targetData)

fixMergeColumns([data], pairs)

We are going to train our model to predict whether the position ends up to a goal or not so we need to convert our goal and not goal situations to binary tags.

In [ ]:
replacement_dict = {
    'Goal': 'Goal',
    'BlockedShot': 'No Goal',
    'MissedShots': 'No Goal',
    'SavedShot': 'No Goal',
    'ShotOnPost': 'No Goal',
    'OwnGoal': 'No Goal'
}

data['result'] = pd.DataFrame(data['result'].map(replacement_dict))

# Exploring Data

In [ ]:
#data.head(2)
print(data['result'].head(5))

### Target Variable

Our target variable is "result", this feature represents whether a shot is a goal or not.

### Feature Variables

Our feature variables are going to help our model to learn and predict the target variable.

In [ ]:
print(data.drop('result',axis=1).columns.tolist())

# Imbalancedness

Our data has imbalancedness problem which might cause our model to learn features of majority target variables better than the minority target variables, it might cause inaccurate predictions.

In [ ]:
print(data['result'].value_counts())

In [ ]:
X = data.drop('result', axis=1)
Y = data['result']
print(Y.head(2))
X = pd.get_dummies(X)

data = pd.concat([pd.DataFrame(Y),pd.DataFrame(X)], axis=1)

In [ ]:
data_shuffled = data.sample(frac=1, random_state=42)
half_length = len(data_shuffled) // 2
df_half1 = data_shuffled.iloc[:half_length]
df_half2 = data_shuffled.iloc[half_length:]

data = df_half1
validation_data = df_half2

We are going to use SMOTE method for oversampling to fix the gap between minority and majority target variables. This method uses clustering methods to create new observations based on original ones.

In [ ]:
data['result'] = data['result'].map({'Goal': 1, 'No Goal': 0})
validation_data['result'] = validation_data['result'].map({'Goal': 1, 'No Goal': 0})

In [ ]:
Y_validation = validation_data['result']
x_validation = validation_data.drop('result', axis=1)

X = pd.DataFrame(data.drop('result', axis=1))
Y = pd.DataFrame(data['result'])
print(data.result.head(5))


In [476]:

sm = SMOTE(k_neighbors=200,random_state=42,n_jobs=-1)
x_res, y_res = sm.fit_resample(X, Y)

# concat the resampled data
data_res = pd.concat([pd.DataFrame(y_res), pd.DataFrame(x_res)], axis=1)
print(data_res.result.value_counts())

scaler = MinMaxScaler()

x_res = scaler.fit_transform(x_res)

X_validation = pd.DataFrame(scaler.fit_transform(x_validation))

Y_validation = pd.DataFrame(validation_data['result'])



result
0    10478
1    10478
Name: count, dtype: int64


# Model

In [477]:
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.25, random_state=42, stratify=y_res)

We have found the best model to use is Random Forest after comparing Decision Tree, Logistic Regression,XGBoost and Random Forest.

In [494]:
model = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

5-Fold Cross Validation score:

In [ ]:
print(cross_val_score(model, x_train, y_train, cv=5, scoring='balanced_accuracy').mean())

### Train - Test Accuracy Comparison to Check Overfitting

Our model seems to have learned train set perfectly, we might have suspected of overfitting much more than the current situation if the test accuracies were bad but accuracy and balanced accuracy scores on train and test sets are very close to each other. Our model is good to go, but we are going to check if hyperparameter tuning or automl could improve our performance.

In [489]:
test_accuracy = accuracy_score(y_test, y_pred)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)

train_accuracy = accuracy_score(y_train, model.predict(x_train))
train_balanced_accuracy = balanced_accuracy_score(y_train, model.predict(x_train))
train_confusion_matrix = confusion_matrix(y_train, model.predict(x_train))

In [491]:
print('Test Accuracy: {}\nTrain Accuracy: {}\n'.format(test_accuracy, train_accuracy))
print('Test Balanced Accuracy: {}\nTrain Balanced Accuracy: {}\n'.format(test_balanced_accuracy, train_balanced_accuracy))
print('Test Confusion Matrix: \n{}\n\nTrain Confusion Matrix: \n{}'.format(test_confusion_matrix, train_confusion_matrix))

Test Accuracy: 0.9646879175415155
Train Accuracy: 1.0

Test Balanced Accuracy: 0.964684965125667
Train Balanced Accuracy: 1.0

Test Confusion Matrix: 
[[2568   52]
 [ 133 2486]]

Train Confusion Matrix: 
[[7858    0]
 [   0 7859]]


### Hyperparameter Tuning

We are going to use the Halving Random Search method and 5-Fold Cross Validation together to find the best hyperparameters, evaluating our iterations by balanced accuracy scores.

The Halving Random Search method uses an elimination system as its base idea: the best of the two compared hyperparameter sets rises above on the leaderboard, and we get the best hyperparameters after the final round. Another pro of Halving Random Search is that it is much faster than Random Search.

In [ ]:
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=50, stop=5000, num=10)],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [int(x) for x in np.linspace(2, 50)],
    'min_samples_split': [int(x) for x in np.linspace(2, 5)],
    'min_samples_leaf': [int(x) for x in np.linspace(2, 5)],
    'bootstrap': [True, False],
}

halving = HalvingRandomSearchCV(model, param_grid, factor=3, resource='n_samples', max_resources=1000, random_state=42, verbose=0,scoring='balanced_accuracy', n_jobs=-1)
halving.fit(x_train, y_train)
print("Best Params:{}/nBest Balanced Accuracy:{}".format(halving.best_params_,halving.best_score_))

We did not get what we wanted from hyperparameter tuning, our balanced accuracy decreased and it might not be ideal to consume more time to random searching, so let's see how AutoML is going to work.

In [ ]:
h2o.init()

In [ ]:
hf = h2o.H2OFrame(pd.concat([pd.DataFrame(x_res), pd.DataFrame(y_res)], axis=1))

train_hf, test_hf = hf.split_frame(ratios=[0.75], seed = 1)

### AutoML

We are going to use the H2O library because of its ease of use. We are going to set the maximum runtime to 300 seconds so our process won't consume too much time. AutoML is going to use 5-Fold Cross Validation by default and compare balanced accuracies and more metrics to choose the best model for our data.

In [ ]:
aml = H2OAutoML(max_models = 20,
                balance_classes=True,
		seed =1,max_runtime_secs=600,verbosity='none')

aml.train(training_frame = train_hf, y = 'result')

AutoML found the best model to be Gradient Boosting Machines. This algorithm is based on the idea of creating an ensemble of weak learners, typically decision trees, in a sequential manner. Each new model attempts to correct the errors of the previous models.

Our new model's performance is almost the same as vanilla Random Forest. However, AutoML checked more metrics to validate that this is the right model for our data. Both models are black box models, so it does not affect us which one we choose for the sake of interpretability. We can trust more in the one that AutoML found for its generalizability because it checked more metrics when building the model. So, we are good to go now.

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
warnings.filterwarnings('ignore')
preds = aml.leader.predict(test_hf)

y_test_gbm = test_hf['result'].as_data_frame().values.flatten()
y_pred_gbm = preds['predict'].as_data_frame().values.flatten()

balanced_acc = balanced_accuracy_score(y_test_gbm, y_pred_gbm)
print("Balanced Accuracy Score: ", balanced_acc)

# Validation of the Model

We are going to test our model with a new, unseen data.

In [ ]:
warnings.filterwarnings('ignore')

validation_x_hf = h2o.H2OFrame(pd.DataFrame(X_validation))
validation_y_hf = h2o.H2OFrame(pd.DataFrame(Y_validation))

                               
preds_validation = aml.leader.predict(validation_x_hf)

y_validation_test_gbm = validation_y_hf['result'].as_data_frame().values.flatten()
y_validation_pred_gbm = preds_validation['predict'].as_data_frame().values.flatten()

balanced_acc = balanced_accuracy_score(y_validation_test_gbm, y_validation_pred_gbm)
print("Balanced Accuracy Score: ", balanced_acc)

print(confusion_matrix(y_validation_test_gbm, y_validation_pred_gbm))

In [493]:
y_validation_pred = model.predict(X_validation)

print(confusion_matrix(Y_validation, y_validation_pred))

print(balanced_accuracy_score(Y_validation, y_validation_pred))

       0         1         2         3         4         5     6         7     \
0  0.834998  0.447619  0.871717  0.494867  0.057366  0.631198   1.0  0.973904   

   8         9     ...  2500  2501  2502  2503  2504  2505  2506  2507  2508  \
0   0.0  0.428571  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   2509  
0   1.0  

[1 rows x 2510 columns]
[[10286   221]
 [  871   304]]
0.6188449038027333
